# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
#Examine dataset
data.info()
data[['race', 'call']].describe()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

,call
count,4870.000000
mean,0.080493
std,0.272079
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
#Mean number of callbacks for each race
data.groupby('race')['call'].mean()

race
b    0.064476
w    0.096509
Name: call, dtype: float32

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

1.  I would use a permutation test here.  The two groups (white and black-sounding names receiving callbacks) would be mixed together randomly and samples drawn to determine whether the observed result that white-sounding names received more callbacks could have been by chance.  The Central Limit Theorem would be applicable here since the names were assigned at random to the resumes and the sample size is greater than 30 (4,870).  

2.  The null hypothesis is that there is no difference in the number of callbacks received from employers based on name.  The alternative hypothesis is that the number of callbacks for white-sounding names is greater.  

In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

In [9]:
# Your solution to Q3 here
from scipy import stats

#Set random seed
np.random.seed(7)

#Define permutation functions
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)
    
    diff = np.abs(diff)

    return diff


white_calls = data[(data['race'] == 'w')]['call']

black_calls = data[(data['race'] == 'b')]['call']

# Draw 1 permutation replicate: perm_sample
perm_sample = draw_perm_reps(white_calls, black_calls, diff_of_means, size=1)

z_critical = stats.norm.ppf(q = 0.975)

margin_of_error = z_critical * np.sqrt((np.sum(white_calls)/len(data) * np.sum(black_calls)/len(data))/len(data))

confidence_interval = (perm_sample[0] - margin_of_error, perm_sample[0] + margin_of_error)

print("Margin of error: ", margin_of_error)

print("Confidence Interval is between", confidence_interval[0], "and", confidence_interval[1], "mean number of callbacks")


# Compute difference of mean call backs: empirical_diff_means
empirical_diff_means = diff_of_means(white_calls, black_calls)


# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(white_calls, black_calls, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value =', p)


Margin of error:  0.0011077414497
Confidence Interval is between 0.00382039293207 and 0.00603587583147 mean number of callbacks
p-value = 0.0


In [10]:
#T-Test Approach
from scipy import stats

print("White sample size: ", len(white_calls), " Black sample size: ", len(black_calls))

a = 0.05

stat, p = stats.ttest_ind(white_calls, black_calls)

print('p-value =', p)

if p < a:
    print("Reject null hypothesis.  Names may have an effect on the number of callbacks.")
else:
    print("Fail to reject null hypothesis. Names have no effect on the number of callbacks.")
    
  

White sample size:  2435  Black sample size:  2435
p-value = 3.94080210313e-05
Reject null hypothesis.  Names may have an effect on the number of callbacks.


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

\#4. Using a significance level of $\alpha$ = 0.05, it appears that the p-value is sufficiently low under both the bootstrap test and the t-test that the null hypothesis that the type of names has no impact on the number of employer callbacks may be rejected.  

\#5. Race/name is not necessarily the most important factor in callback success.  Many other variables are present in this dataset that should also be tested for such as education, yearsexp (years of experince) and specialskills.  A correlation test might be performed between these variables and call to see which has the strongest correlation.  

In [11]:
#Find most strongly correlated variable with callbacks

greatest_factor = data.corr()['call'][data.corr()['call'] == data.corr()['call'].drop(labels='call').max()].index[0]
correlation = data.corr()['call'].drop(labels='call').max()

print("The greatest factor in determining employer callbacks is", greatest_factor, "with a correlation coefficient of", correlation)


The greatest factor in determining employer callbacks is specialskills with a correlation coefficient of 0.11107431271


It therefore appears that whether or not a candidate receives a callback from an employer is most strongly correlated with whether the candidate has special skills.  